     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=1ab0272f853e5f544660e19d346dcd8b2cb1018e8885173193e8535325ce872e
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [2]:
import pandas as pd

def read_conll(path):
    data = []
    with open(path, encoding='utf-8') as f:
        tokens, labels = [], []
        for line in f:
            if line.strip():
                token, label = line.strip().split()
                tokens.append(token)
                labels.append(label)
            else:
                if tokens:
                    data.append({'tokens': tokens, 'ner_tags': labels})
                    tokens, labels = [], []
    return pd.DataFrame(data)

df = read_conll("/amharic_ner.conll")
df.head()


,tokens,ner_tags
0,"[Leather, cOmfOrt, Made, in, Vietnam, Master, ...","[B-PRODUCT, I-PRODUCT, O, O, O, O, O, O, O, O,..."
1,"[HOBBY, LOBBY, 3, in, 1, ChOpper, and, Garlic,...","[B-PRODUCT, I-PRODUCT, O, O, O, O, O, O, O, O,..."
2,"[POrtable, Sealing, Machine, ButtOn, Switch, ➡...","[B-PRODUCT, I-PRODUCT, O, O, O, O, O, O, O, O,..."
3,"[., ., ., ., ., ., ., ., ., ., ., ., ., ., ., ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [3]:
from datasets import Dataset

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2)


In [21]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "Davlan/afro-xlmr-base"  # ↪ use correct hyphens
# or
# model_name = "rasyosef/bert-small-amharic"

tokenizer = AutoTokenizer.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [22]:
unique_tags = set(tag for row in df["ner_tags"] for tag in row)
label2id = {tag: i for i, tag in enumerate(sorted(unique_tags))}
id2label = {v: k for k, v in label2id.items()}
number_of_labels = len(label2id)


In [24]:

 # from transformers import TokenizerFast # Remove this line

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                # This part of the original code has a potential issue.
                # If the current word_idx is the same as the previous_word_idx,
                # it means this token is a subword of the previous word.
                # For NER, you typically only label the first subword of a word.
                # The label for subsequent subwords should usually be -100 or
                # the same label as the first subword depending on the desired
                # behavior and dataset annotation guidelines.
                # The original code re-applies the same label to all subwords.
                # A common practice is to only label the first token of a word
                # and set -100 for subsequent subword tokens.
                # We will keep the original logic for now as the error was an Import Error,
                # but this is a potential area for refinement.
                 label_ids.append(label2id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [28]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./ner-model",
    eval_strategy="epoch",  # Changed from evaluation_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [30]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "Davlan/afro-xlmr-base"  # ↪ use correct hyphens
# or
# model_name = "rasyosef/bert-small-amharic"

tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model for token classification
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=number_of_labels, id2label=id2label, label2id=label2id)

config.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification
from datasets import Dataset
import pandas as pd
import torch # Import torch

# Assuming df, label2id, id2label, number_of_labels, tokenizer, dataset are already defined from previous steps

# Define the tokenization and alignment function
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                # Keep the label for subsequent subwords, consistent with the original logic
                 label_ids.append(label2id[label[word_idx]])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Apply the tokenization function to the dataset
tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

# Define training arguments
args = TrainingArguments(
    output_dir="./ner-model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
)

# Load the model
model_name = "Davlan/afro-xlmr-base"
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=number_of_labels, id2label=id2label, label2id=label2id)

# Instantiate the data collator for token classification
# This collator will handle padding of both inputs and labels
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)


# Instantiate the Trainer with the data collator
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer, # tokenizer is still needed for other trainer functionalities
    data_collator=data_collator # Add the data collator here
)

# Start training
trainer.train()

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-34-3115009857.py:53: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,No log,1.963219
2,No log,1.871618
3,No log,1.805649
4,No log,1.762564
5,No log,1.740920


TrainOutput(global_step=5, training_loss=1.8971952438354491, metrics={'train_runtime': 187.5749, 'train_samples_per_second': 0.08, 'train_steps_per_second': 0.027, 'total_flos': 2304312865470.0, 'train_loss': 1.8971952438354491, 'epoch': 5.0})

In [37]:
import numpy as np
from sklearn.metrics import classification_report # Ensure classification_report is imported

predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
preds = np.argmax(predictions, axis=2)

# Flatten the true_labels and pred_labels lists
true_labels = [id2label[l] for label in labels for l in label if l != -100]
pred_labels = [id2label[p] for pred, label in zip(preds, labels) for (p, l) in zip(pred, label) if l != -100]

print(classification_report(true_labels, pred_labels))

              precision    recall  f1-score   support

       B-LOC       0.03      0.06      0.04        17
     B-PRICE       0.00      0.00      0.00         1
   B-PRODUCT       0.00      0.00      0.00         4
       I-LOC       0.00      0.00      0.00         2
     I-PRICE       0.00      0.00      0.00         1
   I-PRODUCT       0.00      0.00      0.00         2
           O       0.92      0.85      0.89       226

    accuracy                           0.77       253
   macro avg       0.14      0.13      0.13       253
weighted avg       0.83      0.77      0.80       253



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [38]:
trainer.save_model("amharic-ner-finetuned")
tokenizer.save_pretrained("amharic-ner-finetuned")


('amharic-ner-finetuned/tokenizer_config.json',
 'amharic-ner-finetuned/special_tokens_map.json',
 'amharic-ner-finetuned/sentencepiece.bpe.model',
 'amharic-ner-finetuned/added_tokens.json',
 'amharic-ner-finetuned/tokenizer.json')